In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision
import torchvision.utils as vutils
from torch.autograd import Variable
from tqdm.notebook import tqdm_notebook
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.transforms as transforms
import random
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
from PIL import Image
import cv2
import time
from tqdm.notebook import tqdm_notebook
import time
import os, shutil
import matplotlib.animation as animation
from mpl_toolkits.axes_grid1 import ImageGrid
import sys, time
from nibabel.testing import data_path
import nibabel as nib
from PIL import Image
import scipy.ndimage as ndi
import itertools
import time
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding
import tqdm
import nltk
nltk.download('punkt')
from sklearn.metrics import roc_auc_score

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
device = torch.device("cuda")

True
1


In [3]:
data = pd.read_csv('/kaggle/input/dataset/FinalDataset.csv')
data.head()

,sentence,class
0,What city in the United States has the highest...,0
1,"At work, wishing I was out on the boat",0
2,A smile is a curve that sets everything straig...,0
3,Does sleep quality mediate the association bet...,0
4,What city was found on the west bank of the ri...,0


In [4]:
data['class'].value_counts()

0    400015
1    376930
Name: class, dtype: int64

In [5]:
train_data, test_data = train_test_split(data, test_size=0.1, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.1, random_state=42)

In [6]:
# CREATE DATASET CLASS FOR DATALOADERS
class Dataset(Dataset):
    def __init__(self, dataframe):
        self.texts = dataframe['sentence'].values.tolist()
        self.labels = dataframe['class'].values.tolist()
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        X = self.texts[idx]
        y = self.labels[idx]
        X = X.lower()
        tokens = nltk.word_tokenize(X, language="english")
        X = " ".join(tokens)
        X = X.strip()
        
        return X, y

In [7]:
def collate_fn(batch):    
    x, y = [], []
    for text, label in batch:
        x.append(text)
        y.append(label)
    padded_text = tokenizer(x, padding=True, truncation=True, return_tensors='pt')
    
    return padded_text, torch.FloatTensor(y)

In [8]:
class transformer(nn.Module):
    def __init__(self, base_model):
        super(transformer, self).__init__()

        self.bert = base_model
        self.fc1 = nn.Linear(768, 32)
        self.fc2 = nn.Linear(32, 1)

        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, input_ids, attention_mask):
        bert_out = self.bert(input_ids=input_ids,
                             attention_mask=attention_mask)[0][:, 0]
        x = self.fc1(bert_out)
        x = self.relu(x)
        
        x = self.fc2(x)
        x = self.sigmoid(x)

        return x

In [9]:
torch.manual_seed(0)

BERT_MODEL = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(BERT_MODEL)
base_model = AutoModel.from_pretrained(BERT_MODEL)

model = transformer(base_model)
model = model.to(device)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# INFERENCE USING STORED MODEL
dataloader = DataLoader(Dataset(test_data), batch_size=16, num_workers=2, collate_fn=collate_fn)
model = torch.load('/kaggle/input/roberta-model/best_model.pt')
model.eval()
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
total_acc_test = 0
total_loss_test = 0
for test_input, test_label in dataloader:
    attention_mask = test_input['attention_mask'].to(device)
    input_ids = test_input['input_ids'].squeeze(1).to(device)

    test_label = test_label.to(device)

    output = model(input_ids, attention_mask)

    loss = criterion(output, test_label.float().unsqueeze(1))

    total_loss_test += loss.item()

    pred = (output >= 0.5).int()
    label = test_label.unsqueeze(1)
    auc_roc = roc_auc_score(label.cpu(), pred.cpu())
    total_acc_test += auc_roc
    
print("Test Loss:", total_loss_test/len(dataloader))
print("Test AUCROC:", total_acc_test/len(dataloader))

In [10]:
data = {}
count = 0
errors = []
model = torch.load('/kaggle/input/roberta-model/best_model.pt')
model.eval()
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
def getRangeRows(df, x, y):
    df['word_lengths'] = df['sentence'].apply(lambda x: len(x.split(" ")))
    df = df[(df['word_lengths'] >= x) & (df['word_lengths'] <= y)]
    return df
start = 10
while start != 200:
    if start == 195:
        newDf = getRangeRows(test_data, start, 200)
    else:
        newDf = getRangeRows(test_data, start, start + 4)
    start += 5
    count = 0
    # INFERENCE USING STORED MODEL
    dataloader = DataLoader(Dataset(newDf), batch_size=16, num_workers=2, collate_fn=collate_fn)
    total_acc_test = 0
    total_loss_test = 0
    for test_input, test_label in dataloader:
        attention_mask = test_input['attention_mask'].to(device)
        input_ids = test_input['input_ids'].squeeze(1).to(device)

        test_label = test_label.to(device)

        output = model(input_ids, attention_mask)

        loss = criterion(output, test_label.float().unsqueeze(1))

        total_loss_test += loss.item()

        pred = (output >= 0.5).int()
        label = test_label.unsqueeze(1)
        try:
            auc_roc = roc_auc_score(label.cpu(), pred.cpu())
        except:
            count += 1
            key = str(start) + '-' + str(start + 4)
            errors.append([key, count])
            continue
        total_acc_test += auc_roc
        
    key = str(start) + '-' + str(start + 4)
    try:
        test_loss = total_loss_test/(len(dataloader) - count)
        test_aucroc = total_acc_test/(len(dataloader) - count)
        print("For range of text length:", key)
        print("Test Loss:", test_loss)
        print("Test AUCROC:", test_aucroc)
        print()
    except:
        continue
    data[key] = test_aucroc

For range of text length: 15-19
Test Loss: 0.1575334829810976
Test AUCROC: 0.9334783256599406

For range of text length: 20-24
Test Loss: 0.12294149855628109
Test AUCROC: 0.9529911097168904

For range of text length: 25-29
Test Loss: 0.10255630496321637
Test AUCROC: 0.9611482968766455

For range of text length: 30-34
Test Loss: 0.08488206159668912
Test AUCROC: 0.9681443646736487

For range of text length: 35-39
Test Loss: 0.08799995364241135
Test AUCROC: 0.9623507493877868

For range of text length: 40-44
Test Loss: 0.11012367802976193
Test AUCROC: 0.9535503895197777

For range of text length: 45-49
Test Loss: 0.12733520623107716
Test AUCROC: 0.9558049120549125

For range of text length: 50-54
Test Loss: 0.12327423597853936
Test AUCROC: 0.9583351075154355

For range of text length: 55-59
Test Loss: 0.1009333633849806
Test AUCROC: 0.9621628088464823

For range of text length: 60-64
Test Loss: 0.08218827603969914
Test AUCROC: 0.9630297409143563

For range of text length: 65-69
Test Loss: